In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report

# Define RandomForestModel class
class RandomForestModel:
    def __init__(self):
        self.model = RandomForestClassifier(n_estimators=100, random_state=42)

    def load(self, historic_file, prediction_file):
        self.historic_data = pd.read_csv(historic_file)
        self.prediction_data = pd.read_csv(prediction_file)

    def preprocess(self):
        self.historic_data.fillna(0, inplace=True)
        self.prediction_data.fillna(0, inplace=True)

        self.historic_data = pd.get_dummies(self.historic_data, columns=['category', 'main_promotion', 'color'])
        self.prediction_data = pd.get_dummies(self.prediction_data, columns=['category', 'main_promotion', 'color'])

    def train(self):
        X = self.historic_data.drop(columns=['success_indicator', 'item_no'])
        y = self.historic_data['success_indicator']
        self.model.fit(X, y)

    def test(self):
        X_test = self.historic_data.drop(columns=['success_indicator', 'item_no'])
        self.y_test = self.historic_data['success_indicator']
        y_pred = self.model.predict(X_test)
        self.accuracy = accuracy_score(self.y_test, y_pred)
        print("Random Forest Model Evaluation:")
        print("Accuracy:", self.accuracy)
        print(classification_report(self.y_test, y_pred))

    def predict(self):
        predictions = self.model.predict(self.prediction_data.drop(columns='item_no'))
        prediction_df = self.prediction_data.copy()
        prediction_df['prediction'] = predictions
        return prediction_df

# Define ANNModel class
class ANNModel:
    def __init__(self):
        self.model = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000)

    def load(self, data):
        self.data = data

    def preprocess(self):
        self.data.fillna(0, inplace=True)
        self.data = pd.get_dummies(self.data, columns=['category', 'main_promotion', 'color'])
        scaler = StandardScaler()
        self.data[['stars']] = scaler.fit_transform(self.data[['stars']])

    def train(self):
        X = self.data.drop(columns=['success_indicator', 'item_no'])
        y = self.data['success_indicator']
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        self.model.fit(self.X_train, self.y_train)

    def test(self):
        y_pred = self.model.predict(self.X_test)
        self.accuracy = accuracy_score(self.y_test, y_pred)
        print("ANN Model Evaluation:")
        print("Accuracy:", self.accuracy)
        print(classification_report(self.y_test, y_pred))

    def predict(self, new_data):
        new_data.fillna(0, inplace=True)
        new_data = pd.get_dummies(new_data, columns=['category', 'main_promotion', 'color'])
        scaler = StandardScaler()
        new_data[['stars']] = scaler.fit_transform(new_data[['stars']])
        predictions = self.model.predict(new_data.drop(columns='item_no'))
        return predictions

# Load data
historic_data = pd.read_csv("historic.csv")
prediction_data = pd.read_csv("prediction_input.csv")

# RandomForestModel
rf_model = RandomForestModel()
rf_model.load("historic.csv", "prediction_input.csv")
rf_model.preprocess()
rf_model.train()
rf_model.test()

# ANNModel
ann_model = ANNModel()
ann_model.load(historic_data)
ann_model.preprocess()
ann_model.train()
ann_model.test()

# Evaluate and compare performance metrics for both models
# Here, we are using accuracy as the primary metric for comparison
rf_accuracy = rf_model.accuracy
ann_accuracy = ann_model.accuracy

print("Random Forest Model Accuracy:", rf_accuracy)
print("ANN Model Accuracy:", ann_accuracy)

# Choose the model with higher accuracy
chosen_model = "Random Forest" if rf_accuracy > ann_accuracy else "ANN"
print("Selected Model:", chosen_model)

# Explanation for model choice
if chosen_model == "Random Forest":
    print("The Random Forest model was chosen due to its higher accuracy compared to the ANN model.")
else:
    print("The ANN model was chosen due to its higher accuracy compared to the Random Forest model.")


Random Forest Model Evaluation:
Accuracy: 0.899625
              precision    recall  f1-score   support

        flop       0.88      0.83      0.85      2815
         top       0.91      0.94      0.92      5185

    accuracy                           0.90      8000
   macro avg       0.90      0.88      0.89      8000
weighted avg       0.90      0.90      0.90      8000

ANN Model Evaluation:
Accuracy: 0.84875
              precision    recall  f1-score   support

        flop       0.83      0.73      0.77       571
         top       0.86      0.92      0.89      1029

    accuracy                           0.85      1600
   macro avg       0.84      0.82      0.83      1600
weighted avg       0.85      0.85      0.85      1600

Random Forest Model Accuracy: 0.899625
ANN Model Accuracy: 0.84875
Selected Model: Random Forest
The Random Forest model was chosen due to its higher accuracy compared to the ANN model.
